# Linear Models with SparkR: uses and present limitations

[**Introduction to Apache Spark with R by J. A. Dianes**](https://github.com/jadianes/spark-r-notebooks)

In this notebook we will use SparkR machine learning capabilities in order to predict property value in relation to other variables in the [2013 American Community Survey](http://www.census.gov/programs-surveys/acs/data/summary-file.html) dataset. The whole point of R on Spark is to introduce Spark scalability into R data analysis pipelines. With this idea in mind, we have seen how [SparkR](http://spark.apache.org/docs/latest/sparkr.html) introduces data types and functions that are very similar to what we are used to when using regular R libraries. The next step in our series of notebooks will deal with its machine learning capabilities. While building a linear model we also want to check the significance of each of the variables involved in building such a predictor for property value.

## Creating a SparkSQL context and loading data

In order to explore our data, we first need to load it into a SparkSQL data frame. But first we need to init a SparkSQL context. The first thing we need to do is to set up some environment variables and library paths as follows. Remember to replace the value assigned to `SPARK_HOME` with your Spark home folder.  

In [1]:
# Set Spark home and R libs
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var, window

The following objects are masked from ‘package:base’:

    as.data.frame, colnames, colnames<-, drop, endsWith, intersect,
    rank, rbind, sample, startsWith, subset, summary, transform, union



Now we can load the `SparkR` library as follows.

In [2]:
sqlContext <- sparkR.session(master = "local[*]", sparkConfig = list(spark.driver.memory = "2g") ,sparkPackages = "com.databricks:spark-csv_2.11:1.2.0")


Spark package found in SPARK_HOME: /opt/spark


Launching java with spark-submit command /opt/spark/bin/spark-submit  --packages com.databricks:spark-csv_2.11:1.2.0 --driver-memory "2g" sparkr-shell /tmp/RtmpPbuhf2/backend_port3e37263c1311 


And now we can initialise the Spark context as [in the official documentation](http://spark.apache.org/docs/latest/sparkr.html#starting-up-sparkcontext-sqlcontext). In our case we are use a standalone Spark cluster with one master and seven workers. If you are running Spark in local node, use just `master='local'`. Additionally, we require a Spark package from Databricks to read CSV files (more on this in the [previous notebook](https://github.com/jadianes/spark-r-notebooks/blob/master/notebooks/nb1-spark-sql-basics/nb1-spark-sql-basics.ipynb)). 

In [3]:
#sc <- sparkR.init(master='spark://169.254.206.2:7077', sparkPackages="com.databricks:spark-csv_2.11:1.2.0")

And finally we can start the SparkSQL context as follows.

In [4]:
#sqlContext <- sparkRSQL.init(sc)

Now that we have our SparkSQL context ready, we can use it to load our CSV data into data frames. We have downloaded our [2013 American Community Survey dataset](http://www.census.gov/programs-surveys/acs/data/summary-file.html) files in [notebook 0](https://github.com/jadianes/spark-r-notebooks/tree/master/notebooks/nb0-starting-up/nb0-starting-up.ipynb), so they should be stored locally. Remember to set the right path for your data files in the first line, ours is `/nfs/data/2013-acs/ss13husa.csv`.  

In [5]:
housing_a_file_path <- file.path('', 'home','spark','pu_workshop','data','2013-acs','ss13husa.csv')
housing_b_file_path <- file.path('', 'home','spark','pu_workshop','data','2013-acs','ss13husa.csv')

Now let's read into a SparkSQL dataframe. We need to pass four parameters in addition to the `sqlContext`:  

- The file path.  
- `header='true'` since our `csv` files have a header with the column names. 
- Indicate that we want the library to infer the schema.  
- And the source type (the Databricks package in this case). 

And we have two separate files for both, housing and population data. We need to join them.

In [6]:
housing_a_df <- read.df(
    #sqlContext, 
                        housing_a_file_path, 
                        header='true', 
                        source = "com.databricks.spark.csv", 
                        inferSchema='true')

In [7]:
housing_b_df <- read.df(
    #sqlContext, 
                        housing_b_file_path, 
                        header='true', 
                        source = "com.databricks.spark.csv", 
                        inferSchema='true')

In [8]:
housing_df <- rbind(housing_a_df, housing_b_df)

Let's check that we have everything there by counting the files and listing a few of them.

In [9]:
nrows <- nrow(housing_df)
nrows

[1] 1512130

In [10]:
head(housing_df)

RT,SERIALNO,DIVISION,PUMA,REGION,ST,ADJHSG,ADJINC,WGTP,NP,⋯,wgtp71,wgtp72,wgtp73,wgtp74,wgtp75,wgtp76,wgtp77,wgtp78,wgtp79,wgtp80
H,84,6,2600,3,1,1000000,1007549,0,1,⋯,0,0,0,0,0,0,0,0,0,0
H,154,6,2500,3,1,1000000,1007549,51,4,⋯,86,53,59,84,49,15,15,20,50,16
H,156,6,1700,3,1,1000000,1007549,449,1,⋯,161,530,601,579,341,378,387,421,621,486
H,160,6,2200,3,1,1000000,1007549,16,3,⋯,31,24,33,7,7,13,18,23,23,5
H,231,6,2400,3,1,1000000,1007549,52,1,⋯,21,18,37,49,103,38,49,51,46,47
H,286,6,900,3,1,1000000,1007549,76,1,⋯,128,25,68,66,80,26,66,164,88,24


## Preparing our data

We need to convert `ST` (or any other categorical variable) from a numeric variable into a factor.

In [11]:
housing_df$ST <- cast(housing_df$ST, "string")

In [12]:
housing_df$REGION <- cast(housing_df$REGION, "string")

Additionally, we need either to impute values or to remove samples with null values in any of our predictors or desponse. For the response (`VALP`) we will use just those samples with actual values.

In [13]:
housing_with_valp_df <- filter(
    housing_df, 
    isNotNull(housing_df$VALP) 
    & isNotNull(housing_df$TAXP)
    & isNotNull(housing_df$INSP)
    & isNotNull(housing_df$ACR)
)

Let's count the remaining samples.

In [14]:
nrows <- nrow(housing_with_valp_df)
nrows

[1] 818148

## Preparing a train / test data split

We don't have a split function in SparkR, but we can use `sample` in combination with the `SERIALNO` column in order to prepare two sets of IDs for training and testing.

In [15]:
housing_df_test <- sample(housing_with_valp_df,FALSE,0.1)

In [16]:
nrow(housing_df_test)

[1] 81448

In [17]:
test_ids <- collect(select(housing_df_test, "SERIALNO"))$SERIALNO

Unfortunately SparkR doesn't support negative %in% expressions, so we need to do this in two steps. First we add a flag to the whole dataset indicating that a sample belongs to the test set.

In [18]:
housing_with_valp_df$IS_TEST <- housing_with_valp_df$SERIALNO %in% test_ids

And then we use that flag to subset out the train dataset as follows.

In [19]:
housing_df_train <- subset(housing_with_valp_df, housing_with_valp_df$IS_TEST==FALSE)

In [20]:
nrow(housing_df_train)

[1] 663462

However this approach is not very scalable since we are collecting all the test IDs and passing them over to build the new flag column. What if we have a much larger test set? Hopefully futre versions of SparkR will come up with a proper `split` functionality.

## Training a linear model

In order to train a linear model, we call `glm` with the following parameters:
- A formula: sadly, `SparkR::glm()` gives us an error when we pass more than eight variables using `+` in the formula.
- The dataset we want to use to train the model.
- The type of model (gaussian or binomial).  

This doesn't differ much from the usual R `glm` command, although right now is more limited.

The list of variables we have used includes:  

- `RMSP` or number of rooms.
- `ACR` the lot size.
- `INSP` or insurance cost.
- `TAXP` or taxes cost.
- `ELEP` or electricity cost.
- `GASP` or gas cost.
- `ST` that is the state code.
- `REGION` that identifies the region.

In [21]:
model <- glm(
    VALP ~ RMSP + ACR + INSP + TAXP + ELEP + GASP + ST + REGION, 
    data = housing_df_train, 
    family = "gaussian")

In [22]:
# summary(model, signif.stars=TRUE)

ERROR: Error in .local(object, ...): unused argument (signif.stars = TRUE)


Sadly, the current version of `SparkR::summary()` doesn't provide significance starts. That makes model interpretation and selection very difficult. But at least we know how each variables influences a property value. For example, the Midwest region decreases property value, while the West increases it, etc. In order to interpret that we need to have a look at our [data dictionary](http://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMSDataDict13.txt).

In any case, since we don't have significance starts, we can iterate through adding/removing variables and calculating the R2 value. In our case we ended up with the previous model.

## Evaluating our model using the test data

First of all let's obtain the average value for `VALP` that we will use as a reference of a base predictor model.

In [23]:
VALP_mean <- collect(agg(
    housing_df_train, 
    AVG_VALP=mean(housing_df_train$VALP)
))$AVG_VALP

In [24]:
VALP_mean

[1] 267472.8

Let's now predict on our test dataset as follows.

In [25]:
predictions <- predict(model, newData = housing_df_test)

Let's add the squared residuals and squared totals so later on we can calculate [R2](https://en.wikipedia.org/wiki/Coefficient_of_determination).

In [26]:
predictions <- transform(
    predictions, 
    S_res=(predictions$VALP - predictions$prediction)**2, 
    S_tot=(predictions$VALP - VALP_mean)**2)
head(select(predictions, "VALP", "prediction", "S_res", "S_tot"))

VALP,prediction,S_res,S_tot
20000,67811.74,2.285963e+09,6.124278e+10
1843000,391565.54,2.106662e+12,2.482286e+12
150000,234669.17,7.168869e+09,1.379986e+10
225000,303941.54,6.231767e+09,1.803938e+09
250000,186783.93,3.996271e+09,3.052985e+08
40000,83368.65,1.880840e+09,5.174387e+10


In [27]:
nrows_test <- nrow(housing_df_test)
residuals <- collect(agg(
    predictions, 
    SS_res=sum(predictions$S_res),
    SS_tot=sum(predictions$S_tot)
))

In [28]:
residuals

SS_res,SS_tot
6.67335e+15,1.122922e+16


In [29]:
R2 <- 1.0 - (residuals$SS_res/residuals$SS_tot)

In [30]:
R2

[1] 0.4057158

In regression, the R2 coefficient of determination is a statistical measure of how well the regression line approximates the real data points. An R2 of 1 indicates that the regression line perfectly fits the data.

A value of 0.41 doesn't speak very well about our model.

## Conclusions

We still need to improve our model if we really want to be able to predict property values. However there are some limitations in the current SparkR implementation that stop us from doing so. Hopefully these limitations won't be there in further versions. Moreover, we are using a linear model, and the relationships between our predictors and the target variable might not be linear at all.

But right now, in Spark v1.5, the R machine learning capabilities are still very limited. We are missing a few things, such as: 

- Accepting more than 8 variables in formulas using `+`.  
- Having significance stars that help model interpretation and selection.  
- Having other indicators (e.g. R2) in summary objects so we don't have to calculate them ourselves.
- Being able to create more complex formulas (e.g. removing intercepts using 100 + ...) so we don't get negative values, etc.
- Although we have a `sample` method, we are missing a `split` one that we can use to easier have train/test splits.  
- Being able to use more powerful models (or at least models that deal better with non linearities), and not just linear ones.